###### Imports and Pulls

In [1]:
import pandas as pd
import numpy as np
import requests
#import io
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'
import sqlite3 as sq

###### API Key

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

###### Geography Lists

In [3]:
import geodict
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces
censustocommon = geodict.censustocommon

At this time the only variable from the subject tables is "average commute time to work" so there is no data guide. In 2010 and 2020 the variable remains consistent: S0801_C01_046E. 2000 data comes from social explorer; a data download provided by staff Carson Cooper.

In [4]:
#the 2000 data comes from social explorer so it's a read-in

twothousand = pd.read_csv('../../Data Downloads/SocialExplorer_2000_MeanTravelTimetoWork.csv')

In [5]:
twothousand.tail()

,NAME,GEO_ID,Mean Travel Time to Work
472,"Wayne County, Tennessee",0500000US47181,31
473,"Weakley County, Tennessee",0500000US47183,20
474,"White County, Tennessee",0500000US47185,22
475,"Williamson County, Tennessee",0500000US47187,26
476,"Wilson County, Tennessee",0500000US47189,29


In [6]:
twothousand = twothousand.set_index('NAME').transpose()
#rename geographies dict
twothousand = twothousand.rename(columns = censustocommon)
#one extra weird name
twothousand = twothousand.rename(columns = {'Nashville-Davidson (balance), Tennessee': 'Nashville'})
twothousand = twothousand[['Cheatham County', 'Davidson County', 'Dickson County', 'Houston County', 'Humphreys County', 
                           'Maury County', 'Montgomery County', 'Robertson County', 'Rutherford County', 'Stewart County', 
                           'Sumner County', 'Trousdale County', 'Williamson County', 'Wilson County', 'Adams', 'Ashland City', 'Belle Meade', 
                           'Berry Hill', 'Brentwood', 'Burns', 'Cedar Hill', 'Charlotte', 'Clarksville', 'Columbia', 'Coopertown',
                           'Cross Plains', 'Cumberland City', 'Dickson', 'Dover', 'Eagleville', 'Erin', 'Fairview', 'Forest Hills', 'Franklin', 
                           'Gallatin', 'Goodlettsville', 'Greenbrier', 'Hendersonville', 'Kingston Springs', 'La Vergne', 'Lafayette', 'Lebanon', 
                           'McEwen', 'Millersville', 'Mitchellville', 'Mount Juliet', 'Mount Pleasant', 'Murfreesboro', 'Nashville', 'New Johnsonville', 
                           'Nolensville', 'Oak Hill', 'Pegram', 'Pleasant View', 'Portland', 'Ridgetop', 'Slayden', 'Smyrna', 'Spring Hill', 
                           'Springfield', 'Tennessee Ridge', "Thompson's Station", 'Vanleer', 'Watertown', 'Waverly', 'Westmoreland', 'White Bluff', 'White House']]
twothousand = twothousand.transpose().reset_index(drop = False)

In [8]:
twothousand['Source'] = '2000 Social Explorer for ACS5YRSUBJ'

In [10]:
twothousand.head()

,NAME,GEO_ID,Mean Travel Time to Work,Source
0,Cheatham County,0500000US47021,33,2000 Social Explorer for ACS5YRSUBJ
1,Davidson County,0500000US47037,23,2000 Social Explorer for ACS5YRSUBJ
2,Dickson County,0500000US47043,30,2000 Social Explorer for ACS5YRSUBJ
3,Houston County,0500000US47083,32,2000 Social Explorer for ACS5YRSUBJ
4,Humphreys County,0500000US47085,30,2000 Social Explorer for ACS5YRSUBJ


In [74]:
twothousand.to_feather('../../Outputs/CensusBureau_ACS5SUBJ_2000')
conn = sq.connect('../../Outputs/DataBank.db')
twothousand.to_sql('CensusBureau_ACS5SUBJ_2000', conn, if_exists = 'replace', index = False)

68

In [19]:
# twothousand.set_index(['NAME', 'GEO_ID'], inplace = True)
# cols = twothousand.columns
# twothousand.reset_index(drop = False, inplace = True)
# twothousand = twothousand.melt(id_vars = ['NAME', 'GEO_ID'], value_vars = cols, value_name = 'Value', var_name = 'Variable')

In [50]:
#2010 ACS 5 Year Subject Table Estimate
#counties
url_str= 'https://api.census.gov/data/2010/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)                                                              
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = pd.concat([df, kycos], axis = 0)
#ky places call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(censusplaces)]
df = pd.concat([df, places], axis = 0)
#places
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(censusplaces)]
df = pd.concat([df, places], axis = 0)
#state call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = pd.concat([df, state], axis = 0)
#national call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [52]:
twentyten = savename

In [53]:
twentyten['Mean Travel Time to Work'] = twentyten['Mean Travel Time to Work'].astype(float)

In [54]:
twentyten = twentyten.drop(columns = ['StateFIPS', 'GeoFIPS'])

In [55]:
twentyten = twentyten.set_index('NAME').transpose()
#rename geographies dict
twentyten = twentyten.rename(columns = censustocommon)
twentyten = twentyten.transpose().reset_index(drop = False)

In [56]:
twentyten['Source'] = '2006-2010 ACS5YRSUBJ'

In [60]:
twentyten.tail()

,NAME,GEO_ID,Mean Travel Time to Work,Source
69,Westmoreland,1600000US4779420,33.4,2006-2010 ACS5YRSUBJ
70,White Bluff,1600000US4779980,32.6,2006-2010 ACS5YRSUBJ
71,White House,1600000US4780200,28.8,2006-2010 ACS5YRSUBJ
72,Tennessee,0400000US47,23.9,2006-2010 ACS5YRSUBJ
73,US,0100000US,25.2,2006-2010 ACS5YRSUBJ


In [73]:
twentyten.to_feather('../../Outputs/CensusBureau_ACS5SUBJ_2010')
conn = sq.connect('../../Outputs/DataBank.db')
twentyten.to_sql('CensusBureau_ACS5SUBJ_2010', conn, if_exists = 'replace', index = False)

74

In [31]:
# twentyten.set_index(['NAME', 'GEO_ID'], inplace = True)
# cols = twentyten.columns
# twentyten.reset_index(drop = False, inplace = True)
# twentyten = twentyten.melt(id_vars = ['NAME', 'GEO_ID'], value_vars = cols, value_name = 'Value', var_name = 'Variable')

In [61]:
#2021 ACS 5 Year Subject Table Estimate
#counties
url_str= 'https://api.census.gov/data/2021/acs/acs5/subject?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)                                                              
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = pd.concat([df, kycos], axis = 0)
#ky places call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(censusplaces)]
df = pd.concat([df, places], axis = 0)
#places
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(censusplaces)]
df = pd.concat([df, places], axis = 0)
#state call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = pd.concat([df, state], axis = 0)
#national call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'S0801_C01_046E'] #the only variable is average commute
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Mean Travel Time to Work', 'StateFIPS']
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [62]:
twentytwentyone = savename

In [63]:
twentytwentyone['Mean Travel Time to Work'] = twentytwentyone['Mean Travel Time to Work'].astype(float)

In [64]:
twentytwentyone = twentytwentyone.drop(columns = ['StateFIPS', 'GeoFIPS'])

In [65]:
twentytwentyone = twentytwentyone.set_index('NAME').transpose()
#rename geographies dict
twentytwentyone = twentytwentyone.rename(columns = censustocommon)
twentytwentyone = twentytwentyone.transpose().reset_index(drop = False)

In [66]:
twentytwentyone['Source'] = '2017-2021 ACS5YRSUBJ'

In [67]:
twentytwentyone.head()

,NAME,GEO_ID,Mean Travel Time to Work,Source
0,Cheatham County,0500000US47021,33.8,2017-2021 ACS5YRSUBJ
1,Davidson County,0500000US47037,24.9,2017-2021 ACS5YRSUBJ
2,Dickson County,0500000US47043,33.1,2017-2021 ACS5YRSUBJ
3,Houston County,0500000US47083,39.2,2017-2021 ACS5YRSUBJ
4,Humphreys County,0500000US47085,34.6,2017-2021 ACS5YRSUBJ


In [72]:
twentyten.to_feather('../../Outputs/CensusBureau_ACS5SUBJ_2021')
conn = sq.connect('../../Outputs/DataBank.db')
twentytwentyone.to_sql('CensusBureau_ACS5SUBJ_2021', conn, if_exists = 'replace', index = False)

74

In [41]:
# twentytwentyone.set_index(['NAME', 'GEO_ID'], inplace = True)
# cols = twentytwentyone.columns
# twentytwentyone.reset_index(drop = False, inplace = True)
# twentytwentyone = twentytwentyone.melt(id_vars = ['NAME', 'GEO_ID'], value_vars = cols, value_name = 'Value', var_name = 'Variable')